In [5]:
import pandas as pd
import numpy as np
import json
import networkx
from scipy.stats import truncnorm, lognorm
from uuid import uuid5, UUID

In [10]:
BEHAVIOURS_NS = UUID('{bfcb5d54-e392-474c-b81b-26d7fb8cd6cf}')
behaviours_df = pd.DataFrame({
    "name": ["Cycle", "Walk", "PT", "Drive"]
})
behaviours_df["uuid"] = [str(uuid5(BEHAVIOURS_NS, name)) for name in behaviours_df["name"]]
behaviours_df.to_json("out/behaviours.json", orient="records")
cycle = uuid5(BEHAVIOURS_NS, "Cycle")
walk = uuid5(BEHAVIOURS_NS, "Walk")
pt = uuid5(BEHAVIOURS_NS, "PT")
drive = uuid5(BEHAVIOURS_NS, "Drive")

In [7]:
def random_activation():
    a, b = (-1 - 0) / 1, (1 - 0) / 1
    rng = np.random.default_rng()
    if rng.random() <= 0.5:
        return 0.0
    else:
        return truncnorm.rvs(a, b)

In [9]:
BELIEFS_NS = UUID('{6da81031-d93b-4269-9046-598924305983}')
AGENTS_NS = UUID('{e46fc665-ab0d-4cbb-adce-6fad4a0e6455}')
# beliefs on down, b1 positive, b1 negative, b2 positive, etc. No b3 negative.
# cycling mean, cycling sd, walk mean, walk sd, pt mean, pt sd, drive mean, drive sd on across
perceptions_means_and_sds = np.array([
    [0.49, 0.77, 0.44, 0.86, 0.54, 0.68, 0.26, 0.86],
    [-0.39, 0.79, 0.08, 0.76, -0.3, 0.73, -0.07, 0.47],
    [0.56, 0.66, 0.45, 0.76, 0.21, 0.70, 0.58, 0.72],
    [-0.24, 0.71, -0.05, 0.58, -0.04, 0.60, -0.43, 0.79],
    [0.29, 0.73, 0.25, 0.5, 0.5, 0.71, 0.29, 0.95],
    [0.37, 0.75, 0.23, 0.82, 0.36, 0.66, 0.34, 0.85],
    [0.15, 0.99, 0.17, 0.98, -0.13, 0.64, 0, 1],
    [0.4, 0.72, 0.13, 0.88, 0.3, 0.69, 0.43, 0.76],
    [-0.7, 0.48, -0.4, 0.55, -0.43, 0.53, 0, 1.15],
])

# belief x belief. No b3 negative
relationships_means = np.array([
    [1, 1, 0.9, 1.1, 1.1, 1, 1, 1.1, 0.9],
    [1, 1, 1.1, 0.9, 0.9, 1, 1, 0.9, 1.1],
    [0.9, 1.1, 1, 1, 1, 1.1, 0.9, 1.1, 0.9],
    [1.1, 0.9, 1, 1, 1, 0.9, 1.1, 0.9, 1.1],
    [1.1, 0.9, 1, 1, 1, 0.9, 1.1, 1.1, 0.9],
    [1, 1, 1.1, 0.9, 0.9, 1, 1, 1.1, 0.9],
    [1, 1, 0.9, 1.1, 1.1, 1, 1, 0.9, 1.1],
    [1.1, 0.9, 1.1, 0.9, 1.1, 1.1, 0.9, 1, 1],
    [0.9, 1.1, 0.9, 1.1, 0.9, 0.9, 1.1, 1, 1]
])

n_runs = 100
n_agents = 1000
alpha = 0.1
relationship_sd = 0.05
for run in range(n_runs):
    belief_n_ns = uuid5(BELIEFS_NS, f"beliefs_{run}")
    beliefs_n_df = pd.DataFrame({
        "name": [
            "Group 1 Positive",
            "Group 1 Negative",
            "Group 2 Positive",
            "Group 2 Negative",
            "Group 3 Positive",
            "Group 4 Positive",
            "Group 4 Negative",
            "Group 5 Positive",
            "Group 5 Negative",
        ]
    })

    beliefs_n_df["uuid"] = [str(uuid5(belief_n_ns, name)) for name in beliefs_n_df["name"]]

    perceptions = []
    for i in range(len(beliefs_n_df)):
        cycle_a, cycle_b = (-1 - perceptions_means_and_sds[i][0]) / perceptions_means_and_sds[i][1], (1 - perceptions_means_and_sds[i][0]) / perceptions_means_and_sds[i][1]
        walk_a, walk_b = (-1 - perceptions_means_and_sds[i][2]) / perceptions_means_and_sds[i][3], (1 - perceptions_means_and_sds[i][2]) / perceptions_means_and_sds[i][3]
        pt_a, pt_b = (-1 - perceptions_means_and_sds[i][4]) / perceptions_means_and_sds[i][5], (1 - perceptions_means_and_sds[i][4]) / perceptions_means_and_sds[i][5]
        drive_a, drive_b = (-1 - perceptions_means_and_sds[i][6]) / perceptions_means_and_sds[i][7], (1 - perceptions_means_and_sds[i][6]) / perceptions_means_and_sds[i][7]
        
        perceptions.append({
            cycle: truncnorm.rvs(cycle_a, cycle_b, loc=perceptions_means_and_sds[i][0], scale=perceptions_means_and_sds[i][1]),
            walk: truncnorm.rvs(walk_a, walk_b, loc=perceptions_means_and_sds[i][2], scale=perceptions_means_and_sds[i][3]),
            pt: truncnorm.rvs(pt_a, pt_b, loc=perceptions_means_and_sds[i][4], scale=perceptions_means_and_sds[i][5]),
            drive: truncnorm.rvs(drive_a, drive_b, loc=perceptions_means_and_sds[i][6], scale=perceptions_means_and_sds[i][7]),
        })
    beliefs_n_df["perceptions"] = perceptions

    relationships = []
    for i in range(len(beliefs_n_df)):
        relationships_for_belief = {}
        for j in range(len(beliefs_n_df)):
            a, b = (0 - relationships_means[i][j]) / relationship_sd, (100000000 - relationships_means[i][j]) / relationship_sd
            relationships_for_belief[beliefs_n_df.iloc[j]["uuid"]] = truncnorm.rvs(a, b, loc=relationships_means[i][j], scale=relationship_sd)
        relationships.append(relationships_for_belief)
    
    beliefs_n_df["relationships"] = relationships

    beliefs_n_df.to_json(f"out/beliefs_{run}.json", orient="records")

    # Agents -----------------------------------------------------------------------------------------------------------
    agent_n_ns = uuid5(AGENTS_NS, f"agents_{run}")
    agents_n_df = pd.DataFrame({
        "uuid": [str(uuid5(agent_n_ns, str(i))) for i in range(n_agents)]
    })
    agents_n_df["activations"] = [
        [{
            u: random_activation()
            for u in beliefs_n_df["uuid"]
        }] for i in range(n_agents)
    ]

    network = networkx.watts_strogatz_graph(n=n_agents, k=10, p=0.3)
    rng = np.random.default_rng()
    for i in range(n_agents):
        if rng.random() <= 0.8:
            network.add_edge(i, i)
    
    for edge in network.edges:
        network.edges[edge[0], edge[1]]["weight"] = lognorm.rvs(1)

    friends = np.full(n_agents, {})
    for edge in network.edges:
        friends[edge[0]][agents_n_df.iloc[edge[1]]["uuid"]] = network.edges[edge[0], edge[1]]["weight"]
    
    agents_n_df["friends"] = friends

    deltas_mat = rng.normal(loc=1.0-0.001, scale=0.1,size=(n_agents, len(beliefs_n_df)))
    # Ensure fuly positive (very unlikely for this not to be the case)
    deltas_mat = np.abs(deltas_mat) + 0.001

    agents_n_df["deltas"] = [
        {
            beliefs_n_df.iloc[belief_i]["uuid"]: deltas_mat[agent_i, belief_i]
            for belief_i in range(deltas_mat.shape[1])
        } for agent_i in range(deltas_mat.shape[0])
    ]

    prs = []

    for i in range(n_agents):
        prs_for_agent = {}
        for j in range(len(beliefs_n_df)):
            prs_for_belief = {}
            for k in range(len(behaviours_df)):
                mu = perceptions_means_and_sds[j][k*2]
                sigma = perceptions_means_and_sds[j][k*2 + 1] * alpha
                a, b = (-1 - mu) / sigma, (1 - mu) / sigma
                prs_for_belief[behaviours_df.iloc[k]["uuid"]] = truncnorm.rvs(a, b, loc=mu, scale=sigma)
            prs_for_agent[beliefs_n_df.iloc[j]["uuid"]] = prs_for_belief
        prs.append(prs_for_agent)
    
    agents_n_df["performance_relationships"] = prs
    agents_n_df["actions"] = [[] for _i in range(n_agents)]

    agents_n_df.to_json(f"out/agents_{run}.json.zst", compression="zstd", orient="records")

